In [ ]:
# download citrus sinesis sra and convert to fastq
fastq-dump SRR5799051 SRR4240447 SRR4237671

In [ ]:
#install fastqc and unzip the file
sudo wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.8.zip
sudo unzip fastqc_v0.11.8.zip -d /fastQC

In [ ]:
#run fastqc on multiple files
# checks the quality of your data and outputs the result as html
fastqc SRR5799051.fastq SRR4240447.fastq SRR4237671.fastq -o /data/fastQC

In [ ]:
# fastq-dump the split file sra
fastq-dump --split-files SRR5799051 SRR4240447 SRR4237671

In [ ]:
# install trimmoatic
sudo apt-install trimmomatic
# use the generic quality trimm and adapter clip
java -jar trimmomatic-0.39.jar PE SRR4240447_1.fastq SRR4240447_2.fastq pe.SRR4240447_1.fastq ue.SRR4240447_1.fastq pe.SRR4240447_2.fastq ue.SRR4240447_2.fastq ILLUMINACLIP:TruSeq3-P
E.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

java -jar trimmomatic-0.39.jar PE SRR4237671_1.fastq SRR4237671_2.fastq pe.SRR4237671_1.fastq ue.SRR4237671_1.fastq pe.SRR4237671_2.fastq ue.SRR4237671_2.fastq ILLUMINACLIP:TruSeq3-P
E.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

java -jar trimmomatic-0.39.jar PE SRR5799051_1.fastq SRR5799051_2.fastq pe.SRR5799051_1.fastq ue.SRR5799051_1.fastq pe.SRR5799051_2.fastq ue.SRR5799051_2.fastq ILLUMINACLIP:TruSeq3-P
E.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

In [ ]:
# allow access to write, read, and run files in a certain folder
chmod 777 -R {folder}

In [ ]:
#install the updated bwa from github
wget https://github.com/lh3/bwa/releases/download/v0.7.17/bwa-0.7.17.tar.bz2
# get the sweet orange reference genome
scp kduan@130.127.106.25:/RNA/VALENCIA/csi.chromosome.fa /data
# index the reference genome
bwa index csi.chromosome.fa

#rename the paired reads to have identical read names
sed 
#map the samples to the reads
bwa mem -t 40 csi.chromosome.fa pe.SRR4237671_1.fastq pe.SRR4237671_2.fastq > aln-pe.SRR4237671.sam
bwa mem -t 40 csi.chromosome.fa pe.SRR4240447_1.fastq  pe.SRR4240447_2.fastq > aln-pe.SRR4240447.sam
bwa mem -t 40 csi.chromosome.fa pe.SRR5799051_1.fastq  pe.SRR5799051_2.fastq > aln-pe.SRR5799051.sam

In [ ]:
# sort the mapped alignments using samtools
apt-get install samtools

samtools sort -@ 40 -o aln-pe.SRR4237671.bam aln-pe.SRR4237671.sam
samtools sort -@ 40 -o aln-pe.SRR4240447.bam aln-pe.SRR4240447.sam
samtools sort -@ 40 -o aln-pe.SRR5799051.bam aln-pe.SRR5799051.sam

#remove duplicates 
samtools rmdup aln-pe.SRR4237671.bam aln-pe-rm.SRR4237671.bam
samtools rmdup aln-pe.SRR4240447.bam aln-pe-rm.SRR4240447.bam
samtools rmdup aln-pe.SRR5799051.bam aln-pe-rm.SRR5799051.bam

# merge bam files into one
# use bcftools to call the SNPs of the aligned sequences
# must convert bcf file to run call function
bcftools mpileup -q 20 -Q 20 -Ou -o aln-pe-sequence.bcf -f csi.chromosome.fa aln-pe.SRR4237671.sam aln-pe.SRR4240447.sam aln-pe.SRR5799051.sam

In [ ]:
# call snps
bcftools call -c -v -Ov -o aln-pe-snps.vcf aln-pe-reads.bcf

In [ ]:
#filter the reads to ensure better quality(less false positives)
#bcf filter to ensure the entire quality of the mapped reads are >= 50
bcftools filter -e'QUAL<=50' -o aln-pe-snps-filt.vcf aln-pe-snps.vcf

#filter reads that 0.5<=dp<=1.5 avg_cov
# create txt file in same format as given on the genomecov documentation using sed
#find avg_cov of all the bam files
bedtools genomecov -ibam aln-pe-rm.SRR4237671.bam -g csi.txt > covlist423.txt
# find the sum of the cov values 
awk '{total = total + $3}END {print total}' covlist423.txt # = 655889340 
#find length of the txt file to find number of repeats
wc -l covlist423.txt # = 102123

#repeat for the other bam files
bedtools genomecov -ibam aln-pe-rm.SRR4240447.bam -g csi.txt > covlist424.txt
awk '{total = total + $3}END {print total}' covlist424.txt #= 655889340 

bedtools genomecov -ibam aln-pe-rm.SRR5799051.bam -g csi.txt > covlist579.txt


In [1]:
# calculate the avg_cov of the samples 
# go to Pandas allele ipynb

In [ ]:
#subset the vcf file 0.5avgcov <= DP <= 1.5 avgcov
bcftools view -i'DP>=53 && DF<= 159' -o aln1-pe-snps-filt.vcf aln-pe=snps-filt.vcf

#filter the vcf file where SNP are w/in 5bp. ensure that the SNP isn't due to indel (shift)
bcftools filter -g 5 -o aln2-pe-snps-filt.vcf aln1-pe-snps-filt.vcf

In [ ]:
# the variations of alleles have been found
# view the bam and vcf files using software igv
wget http://data.broadinstitute.org/igv/projects/downloads/2.5/IGV_Linux_2.5.3.zip

unzip IGV_Linux_2.5.3.zip

#make the script executable
chmod a+x igv.sh



In [ ]:
#create a corresponding BAI index
samtools index -@ 40 aln-pe-rm.SRR4237671.bam
samtools index -@ 40 aln-pe-rm.SRR4240447.bam
samtools index -@ 40 aln-pe-rm.SRR5799051.bam